# Motivating example

In [1]:
import numpy as np
import numba

In [10]:
def mean_python(a):
    total = 0
    for i in a:
        total += i
    return total / len(a)

def mean_numpy(a):
    return np.mean(a)

mean_numba = numba.jit(mean_python)

Here, I'm using `numba.njit` as a function, but it's more common to see it used as a decroator. `mean_numba = numba.jit(mean_python)` is equivalent to:

```python
@numba.jit
def mean_numba(a):
    total = 0
    for i in a:
        total += i
    return total / len(a)
```

In [7]:
a = np.random.randint(1, 100, 1000000)

In [ ]:
%time mean_python(a)
%time mean_numpy(a)
%time mean_numba(a)

In [ ]:
%timeit mean_python(a)
%timeit mean_numpy(a)
%timeit mean_numba(a)

In [4]:
mean_numba.inspect_types()

# Why is numba fast

Going over the previous example:

For every instance of an object python has to figure out what type it is and how to perform the operation

Numpy is able to skip all this by knowing the container only has one kind of object in it. This way it can just dispach to compiled code which performs a defined set of operations on those objects.

Numba is able to look at python code and figure out what the fast cpu code would be. In practice, this means loops are no longer slow.

# Trying it out

For a first go round, let's try extending the previous example.

Something we often want to calculate along with the mean is the [variance](https://en.wikipedia.org/wiki/Variance).

$$ Var(X) = (\frac{1}{n}\sum_{i=1}^n x_i^2) - \mu^2$$

Where $\mu$ is the mean:

$$ \mu = \frac{1}{n}\sum_{i=1}^2 x_i $$

Without taking using `np.mean` or `np.var`, given a two dimensional numpy array, calculate the mean and variance for each row in a single pass:

In [28]:
@numba.njit
def meanvar(x):
    sum = 0.
    sumsqs = 0.
    n = 0

    for i in x:
        sum += i
        sumsqs += i ** 2
        n += 1

    mean = sum / n
    var = (sumsqs / n) - (mean ** 2)
    return mean, var

In [30]:
a = np.random.normal(1, 3, (100000))

In [37]:
# Test that should pass:
np.allclose(meanvar(a), (np.mean(a), np.var(a)))

True

In [31]:
%timeit meanvar(a)

84.8 µs ± 843 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%timeit np.mean(a), np.var(a)

## A bit more about jit

In [2]:
@numba.vectorize()
def meanvar(x):
    sum = 0.
    sumsqs = 0.
    n = 0

    for i in x:
        sum += i
        sumsqs += i ** 2
        n += 1

    mean = sum / n
    var = (sumsqs / n) - (mean ** 2)
    return mean, var